In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Hyper-parameters 
num_epochs = 5
batch_size = 4
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
#Transform data to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [ ]:
#Data, put in data after .datasets.
train_dataset = torchvision.datasets."put data here"(root='./data', train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets."put data here"(root='./data', train=False,
                                       download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

In [4]:
task_1_labels = ('Informative','Not Informative', 'Don’t know or can’t judge')
task_2_labels = ('Infrastructure and utility damage', 'Vehicle damage', 'Rescue, volunteering, or donation effort',
                 'Injured or dead people', 'Affected individuals', 'Missing or found people', 'Other relevant information',
                 'Not relevant or can’t judge')
task_3_labels = ('Severe building damage', 'Severe bridge damage', 'Severe road damage',
                 'Mild building damage', 'Mild bridge damage', 'Mild road damage',
                 'Little or no damage', 'Dont know or cant judge')
                 

In [5]:
def imshow(img): #SHow the image function if we want to pull a specific image
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [6]:
#Define the convnet (Eventually replae with mobile 2)
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3) #specify the size (outer numbers) and amount (middle number) of filters
        self.pool = nn.MaxPool2d(2, 2) #specify pool size first number is size of pool, second is step size
        self.conv2 = nn.Conv2d(16, 8, 3) #new depth is amount of filters in previous conv layer
        self.fc1 = nn.Linear(224*224*8, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 3) #finial fc layer needs 19 outputs because we have 19 layers

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  #convolve, relu, pool
        x = self.pool(F.relu(self.conv2(x)))  #convolve, relu, pool
        x = x.view(-1,224*224*8)   #flatten 
        x = F.relu(self.fc1(x))    #fully connected, relu           
        x = F.relu(self.fc2(x))    #fully connected, relu            
        x = self.fc3(x)            #output           
        return x


In [4]:
#Train the model
model = ConvNet().to(device)

cross_entropy = nn.CrossEntropyLoss() #cross entropy loss for our loss function
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #stocastic gradient descent for our optimization algorithm

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: 
        # input_layer:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = cross_entropy(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad() #apply optimization func
        loss.backward() #backpropagation
        optimizer.step() #single step updating parameters
        
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

NameError: name 'ConvNet' is not defined

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')